In [ ]:
from numba import cuda
import numpy as np

In [ ]:
@cuda.jit
def stencil(y, x, h, N, thread_count):
    idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x;
    
    # idx = i * N + j
    
    if (idx<thread_count):
        i = idx // N
        j = idx % N
        y[idx] = -4.0*x[idx]
        if (i<N-1):
            y[idx] = y[idx] + x[idx+N] 
        if (i>0):
            y[idx] = y[idx] + x[idx-N]
        if (j>0):
            y[idx] = y[idx] + x[idx-1]
        if (j<N-1):
            y[idx] = y[idx] + x[idx+1]
            
        y[idx] = y[idx] / (h*h)

In [ ]:
N = 1024

h = 0.1

x_h = np.ones(N*N)
y_h = np.zeros(N*N)

x_d = cuda.device_array(N*N, dtype=x_h.dtype)
y_d = cuda.device_array(N*N, dtype=y_h.dtype)

x_d.copy_to_device(x_h)
 
block_size = 1024
grid_size = (N*N + (block_size - 1)) // block_size

stencil[grid_size, block_size](y_d, x_d, h, N, N*N)
cuda.synchronize()
    
y_d.copy_to_host(y_h)

array([-200., -100., -100., ..., -100., -100., -200.])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8c8d4e3f-c0cf-41f5-9788-4fdefe7c0a84' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>